<a href="https://colab.research.google.com/github/sagar9926/ReinforcementLearning/blob/master/TD3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Twin-Delayed DDPG

## Installing the packages

In [0]:
!pip3 install pybullet --upgrade
!pip install gym

     |████████████████████████████████| 83.7MB 64kB/s 
  Created wheel for pybullet: filename=pybullet-2.7.7-cp36-cp36m-linux_x86_64.whl size=95743633 sha256=8b5591287c4f77ab4ade7f1ffdf4fc62f6d47d80b08ed22640a4437908143e65
  Stored in directory: /root/.cache/pip/wheels/95/10/be/0e11971182fb75c5a2f0e00491f35a9aa31bb5824cc7aea6c6
Successfully built pybullet


## Importing the libraries

In [0]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import pybullet_envs
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from gym import wrappers
from torch.autograd import Variable
from collections import deque

## Step 1: Initializing the Experience Replay memory


In [0]:
class ReplayBuffer(object): # since not inherriting from any other class thus passing object

  def __init__(self, max_size=1000000): 
    self.storage = []                             # This is memory itself
    self.max_size = max_size                      
    self.ptr = 0                                  # index of different cells of memory, use to add transition to memory or do some sampling

  def add(self, transition):                      # adds any new transition to the memory
    if len(self.storage) == self.max_size:        # Check if memory has been fully populated
      self.storage[int(self.ptr)] = transition    # if yes then the new transition should be added at the beginning
      self.ptr = (self.ptr + 1) % self.max_size   #  Incrementing the pointer
    else:
      self.storage.append(transition)            # if memory is not fully populated , then just append the new transition 





  def sample(self, batch_size):                 # Sample the transitions from memory and put them in the batch
    ind = np.random.randint(0, len(self.storage), size=batch_size) # generating some random indexes equal to batch size
    batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = [], [], [], [], [] 
    for i in ind: # iterate over the random indexes 

      state, next_state, action, reward, done = self.storage[i] # get the transition present at index i
      batch_states.append(np.array(state, copy=False))
      batch_next_states.append(np.array(next_state, copy=False))
      batch_actions.append(np.array(action, copy=False))
      batch_rewards.append(np.array(reward, copy=False))
      batch_dones.append(np.array(done, copy=False))
      # before we convert batches into torch tensors we have to convert all of them to numpy array, Reshape converts the array into 1d array
    return np.array(batch_states), np.array(batch_next_states), np.array(batch_actions), np.array(batch_rewards).reshape(-1, 1), np.array(batch_dones).reshape(-1, 1)

## Step 2: We build one neural network for the Actor model and one neural network for the Actor target

In [0]:
class Actor(nn.Module):
  
  def __init__(self, state_dim, action_dim, max_action):
    #max action is to clip in case we added too much noise 
    super(Actor, self).__init__() # Activate the inherritance
    self.layer_1 = nn.Linear(state_dim, 400)
    self.layer_2 = nn.Linear(400, 300)
    self.layer_3 = nn.Linear(300, action_dim)
    self.max_action = max_action

  def forward(self, x):
    x = F.relu(self.layer_1(x)) #applying RELU breaks the non linearity
    x = F.relu(self.layer_2(x))
    x = self.max_action * torch.tanh(self.layer_3(x)) # Tanh returns the value between -1 and 1.By multiplying by max action we get continuous 
                                                      # values for actions within the permissible range 
    return x

## Step 3: We build two neural networks for the two Critic models and two neural networks for the two Critic targets

In [0]:
class Critic(nn.Module):
  
  def __init__(self, state_dim, action_dim):
    super(Critic, self).__init__()
    # since we are going to use two critic model at the same time thus as defined below
    # Defining the first Critic neural network
    self.layer_1 = nn.Linear(state_dim + action_dim, 400)
    self.layer_2 = nn.Linear(400, 300)
    self.layer_3 = nn.Linear(300, 1) #output is one as we output single Q value
    # Defining the second Critic neural network
    self.layer_4 = nn.Linear(state_dim + action_dim, 400)
    self.layer_5 = nn.Linear(400, 300)
    self.layer_6 = nn.Linear(300, 1)

  def forward(self, x, u):
    xu = torch.cat([x, u], 1) # Concatenation of State and Action Vertically
    # Forward-Propagation on the first Critic Neural Network
    x1 = F.relu(self.layer_1(xu))
    x1 = F.relu(self.layer_2(x1))
    x1 = self.layer_3(x1)
    # Forward-Propagation on the second Critic Neural Network
    x2 = F.relu(self.layer_4(xu))
    x2 = F.relu(self.layer_5(x2))
    x2 = self.layer_6(x2)
    return x1, x2

  def Q1(self, x, u): # this will be used for the step of gradient ascent of actor model. 
    xu = torch.cat([x, u], 1)
    x1 = F.relu(self.layer_1(xu))
    x1 = F.relu(self.layer_2(x1))
    x1 = self.layer_3(x1)
    return x1

## Steps 4 to 15: Training Process

In [0]:
# Selecting the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Building the whole Training Process into a class

class TD3(object): # since not inherriting from any other class thus passing object as parameter
  
  def __init__(self, state_dim, action_dim, max_action): # In this method we will create all our objects models and optimizers

    #making sure our T3D class can work with any environment
    # Create all our steps on device selected

    self.actor = Actor(state_dim, action_dim, max_action).to(device)  # Trained via GradientAscent
    self.actor_target = Actor(state_dim, action_dim, max_action).to(device) # Trained vis Polyak Averaging
    self.actor_target.load_state_dict(self.actor.state_dict()) #load the actor target with the weights of the actor model
    self.actor_optimizer = torch.optim.Adam(self.actor.parameters()) 


    self.critic = Critic(state_dim, action_dim).to(device)
    self.critic_target = Critic(state_dim, action_dim).to(device)
    self.critic_target.load_state_dict(self.critic.state_dict())
    self.critic_optimizer = torch.optim.Adam(self.critic.parameters())


    self.max_action = max_action

  def select_action(self, state):
    state = torch.Tensor(state.reshape(1, -1)).to(device) #Converting to torch tensor 1D array
    return self.actor(state).cpu().data.numpy().flatten() # Feeding the state to actor model to get the action
    # Force the computation of forward pass on CPU and extract the data and convert to numpy and flatten
    # the output to get 1D array

    # Why numpy -> Torch -> Numpy
    # At some point we need to use neural network for which we need pytorch format but we convert it back to numpy coz at some time we clip
    # the action after adding noise to it and we are adding noise and clipping by numpy

  def train(self, replay_buffer, iterations, batch_size=100, discount=0.99, tau=0.005, policy_noise=0.2, noise_clip=0.5, policy_freq=2):
    
    for it in range(iterations):
      
      # Step 4: We sample a batch of transitions (s, s’, a, r) from the memory

      batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = replay_buffer.sample(batch_size)
      # since we need to fed them to Neural Network thus convert them to Torch tensor
      state = torch.Tensor(batch_states).to(device) # Will go in NN # This is still a batch of states
      next_state = torch.Tensor(batch_next_states).to(device)
      action = torch.Tensor(batch_actions).to(device)
      reward = torch.Tensor(batch_rewards).to(device)
      done = torch.Tensor(batch_dones).to(device)
      
      # Step 5: From the next state s’, the Actor target plays the next action a’
      next_action = self.actor_target(next_state)
      
      # Step 6: We add Gaussian noise to this next action a’ and we clamp it in a range of values supported by the environment

      #Generate the Gaussian noise for each of the elements of the batch of next actions
      noise = torch.Tensor(batch_actions).data.normal_(0, policy_noise).to(device)  # we create a noise tensor of size same size as batch actons 
      # Clipping the gaussian noise
      noise = noise.clamp(-noise_clip, noise_clip)
      # Adding noise and clippind the action 
      next_action = (next_action + noise).clamp(-self.max_action, self.max_action)
      
      # Step 7: The two Critic targets take each the couple (s’, a’) as input and return two Q-values Qt1(s’,a’) and Qt2(s’,a’) as outputs
      target_Q1, target_Q2 = self.critic_target(next_state, next_action)
      
      # Step 8: We keep the minimum of these two Q-values: min(Qt1, Qt2)
      target_Q = torch.min(target_Q1, target_Q2)
      
      # Step 9: We get the final target of the two Critic models, which is: Qt = r + γ * min(Qt1, Qt2), where γ is the discount factor
      # we are training the model for the entire episode
      # min(Qt1, Qt2) this represents the value of the next state
      # and if we are at the end of episode there is no next state and we have to start new episode
      # therefore min(Qt1, Qt2) this doesnt apply any more
      
      target_Q = reward + ((1 - done) * discount * target_Q).detach()
      
      # Step 10: The two Critic models take each the couple (s, a) as input and return two Q-values Q1(s,a) and Q2(s,a) as outputs
      current_Q1, current_Q2 = self.critic(state, action)
      
      # Step 11: We compute the loss coming from the two Critic models: Critic Loss = MSE_Loss(Q1(s,a), Qt) + MSE_Loss(Q2(s,a), Qt)
      critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)
      
      # Step 12: We backpropagate this Critic loss and update the parameters of the two Critic models with a SGD optimizer
      self.critic_optimizer.zero_grad()
      critic_loss.backward()
      self.critic_optimizer.step()
      
      # Step 13: Once every two iterations, we update our Actor model by performing gradient ascent on the output of the first Critic model
      if it % policy_freq == 0: # Delayed part of T3 DDPG model
        actor_loss = -self.critic.Q1(state, self.actor(state)).mean() # Negative sign because we are performing gradient Ascend
        # In above step we don't input here the current action played corresponding to state
        # Differentiate wrt the actor parameters
        self.actor_optimizer.zero_grad()
        actor_loss.backward() # Gradient Ascend step
        self.actor_optimizer.step()
        
        # Step 14: Still once every two iterations, we update the weights of the Actor target by polyak averaging
        for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
          target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
        
        # Step 15: Still once every two iterations, we update the weights of the Critic target by polyak averaging
        for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
          target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
  